<a href="https://colab.research.google.com/github/Marcin19721205/IntroductionToBigDataWSB/blob/main/Zadanie14_MJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Predykcja produkcji piwa

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

import matplotlib.pyplot as plt
import statsmodels.api as sm

from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import plotly.graph_objects as go

from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score


odczytaj dane - w pythonie też są

In [2]:
data = sm.datasets.get_rdataset('ausbeer', 'fpp2', cache=True).data
data['date'] = data['time'].apply(lambda x: datetime(int(x), int((x % 1) * 12) + 1, 1))
data = data.drop(columns=['time'])
data = data.set_index('date')
data.head()

,value
date,
1956-01-01,284
1956-04-01,213
1956-07-01,227
1956-10-01,308
1957-01-01,262


In [3]:
#dekompozycja danych
from statsmodels.tsa.seasonal import seasonal_decompose
# ausbeer jest kwartalny → period=4
result = seasonal_decompose(data['value'], model='additive', period=4)
# wrzucamy wszystko do jednego DataFrame
comp = pd.DataFrame({
    'observed': result.observed,
    'trend':    result.trend,
    'seasonal': result.seasonal,
    'resid':    result.resid
}, index=data.index)


In [6]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    subplot_titles=("Dane surowe", "Trend", "Sezonowość", "Reszta")
)

fig.add_trace(
    go.Scatter(x=comp.index, y=comp['observed'], name='Observed'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=comp.index, y=comp['trend'], name='Trend'),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=comp.index, y=comp['seasonal'], name='Seasonal'),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=comp.index, y=comp['resid'], name='Residual'),
    row=4, col=1
)

fig.update_layout(
    height=800,
    width=950,
    title_text="Dekompozycja szeregu czasowego – wszystkie składowe",
    template="plotly_dark",
    showlegend=False
)

fig.show()


#Regresja liniowa
przygotowanie zmiennych do modelu

In [7]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(data['value'], model='additive', period=4)
trend = result.trend.dropna()


In [8]:

y_trend = trend.values
X_trend = np.arange(len(y_trend)).reshape(-1, 1)


Regresja

In [9]:
from sklearn.linear_model import LinearRegression

reg_trend = LinearRegression()
reg_trend.fit(X_trend, y_trend)

trend_pred = reg_trend.predict(X_trend)


Predykcja

In [10]:
future_steps = 40
X_future = np.arange(len(y_trend), len(y_trend) + future_steps).reshape(-1, 1)
trend_future = reg_trend.predict(X_future)


In [11]:
future_steps = 40
X_future = np.arange(len(y_trend), len(y_trend) + future_steps).reshape(-1, 1)
trend_future = reg_trend.predict(X_future)


Oś czasu dla prognozy

In [12]:
future_dates = pd.date_range(
    start=trend.index[-1] + pd.offsets.QuarterEnd(),
    periods=future_steps,
    freq="QE"
)


In [13]:
#całość kodu
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
import plotly.graph_objects as go

#Dekompozycja
result = seasonal_decompose(data['value'], model='additive', period=4)
trend = result.trend.dropna()

#Przygotowanie X, y
y_trend = trend.values
X_trend = np.arange(len(y_trend)).reshape(-1, 1)

#egresja liniowa
reg = LinearRegression()
reg.fit(X_trend, y_trend)

trend_pred = reg.predict(X_trend)

#Predykcja na przyszłość
future_steps = 40  # 10 lat (40 kwartałów)
X_future = np.arange(len(y_trend), len(y_trend) + future_steps).reshape(-1, 1)
trend_future = reg.predict(X_future)

future_dates = pd.date_range(
    start=trend.index[-1] + pd.offsets.QuarterEnd(),
    periods=future_steps,
    freq='QE'
)

#Równanie i R²
a = reg.coef_[0]
b = reg.intercept_
R2 = r2_score(y_trend, trend_pred)

eq_text = f"y = {a:.4f}·x + {b:.2f}<br>R² = {R2:.4f}"

#Wykres
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=trend.index,
    y=trend.values,
    mode="lines",
    name="Trend (historyczny)",
))

fig.add_trace(go.Scatter(
    x=trend.index,
    y=trend_pred,
    mode="lines",
    name="Regresja liniowa trendu",
    line=dict(color="orange")
))

fig.add_trace(go.Scatter(
    x=future_dates,
    y=trend_future,
    mode="lines",
    name="Prognoza 10 lat (trend)",
    line=dict(color="yellow", dash="dash")
))

#Pole z równaniem
fig.add_annotation(
    x=0.02, y=0.95,
    xref="paper", yref="paper",
    showarrow=False,
    align="left",
    bgcolor="rgba(0,100,0,0.75)",
    bordercolor="white",
    borderwidth=1,
    borderpad=4,
    font=dict(color="white", size=14),
    text=eq_text
)

fig.update_layout(
    template="plotly_dark",
    width=950,
    height=550,
    title="Regresja liniowa TRENDu + prognoza 10 lat + równanie i R²",
    xaxis_title="Data",
    yaxis_title="Trend wartości"
)

fig.show()


Regresja wielomianowa

In [15]:


#REND ---
result = seasonal_decompose(data['value'], model='additive', period=4)
trend = result.trend.dropna()

y_trend = trend.values
X_trend = np.arange(len(y_trend)).reshape(-1, 1)

#regresja wielomianowa
degree = 3  #--<<< rząd wielonmianu

poly = PolynomialFeatures(degree=degree)
X_poly = poly.fit_transform(X_trend)

model_poly = LinearRegression()
model_poly.fit(X_poly, y_trend)

y_poly_pred = model_poly.predict(X_poly)

# Prognoza na 10 lat (x 4 próbki / year)
future_steps = 40
X_future = np.arange(len(y_trend), len(y_trend) + future_steps).reshape(-1, 1)
X_future_poly = poly.transform(X_future)
y_poly_future = model_poly.predict(X_future_poly)

future_dates = pd.date_range(
    start=trend.index[-1] + pd.offsets.QuarterEnd(),
    periods=future_steps,
    freq='QE'
)

#Równanie wielomianowe
coeffs = model_poly.coef_
intercept = model_poly.intercept_

poly_eq = "y = "
for i, c in enumerate(coeffs):
    if i == 0:
        continue
    poly_eq += f"{c:.4f}·x^{i} + "
poly_eq += f"{intercept:.4f}"

R2_poly = r2_score(y_trend, y_poly_pred)

eq_text = f"{poly_eq}<br>R² = {R2_poly:.4f}"

#Wykres ---
fig = go.Figure()

fig.add_trace(go.Scatter(x=trend.index, y=trend.values, mode="lines", name="Trend"))
fig.add_trace(go.Scatter(x=trend.index, y=y_poly_pred, mode="lines", name=f"Poly deg={degree}", line=dict(color="orange")))
fig.add_trace(go.Scatter(x=future_dates, y=y_poly_future, mode="lines", name="Prognoza", line=dict(color="yellow", dash="dash")))

fig.add_annotation(
    x=0.02, y=0.95, xref="paper", yref="paper",
    showarrow=False, align="left",
    bgcolor="rgba(0,100,0,0.75)", bordercolor="white",
    borderwidth=1, borderpad=4,
    font=dict(color="white", size=14),
    text=eq_text
)

fig.update_layout(
    template="plotly_dark",
    width=1000,
    height=550,
    title=f"Regresja wielomianowa (stopień {degree}) na TRENDZIE + prognoza 10 lat",
    xaxis_title="Data", yaxis_title="Trend"
)

fig.show()


#Metoda autoregresyjana AR1

In [16]:

result = seasonal_decompose(data['value'], model='additive', period=4)
trend = result.trend.dropna()

y = trend.values
idx = trend.index

#AR(1): y_t = a1*y_{t-1} + b
X_ar1 = y[:-1].reshape(-1, 1)   # y_{t-1}
y_ar1 = y[1:]                   # y_t

model_ar1 = LinearRegression()
model_ar1.fit(X_ar1, y_ar1)

y_ar1_pred = model_ar1.predict(X_ar1)

a1 = model_ar1.coef_[0]
b1 = model_ar1.intercept_
R2_ar1 = r2_score(y_ar1, y_ar1_pred)

eq_text_ar1 = f"AR(1): yₜ = {a1:.4f}·yₜ₋₁ + {b1:.4f}<br>R² = {R2_ar1:.4f}"

#Prognoza 10 lat (40 kwartałów) – iteracyjnie
future_steps = 40

y_history_ar1 = list(y.copy())
for _ in range(future_steps):
    y_last = y_history_ar1[-1]
    y_next = a1 * y_last + b1
    y_history_ar1.append(y_next)

y_future_ar1 = np.array(y_history_ar1[-future_steps:])

future_dates = pd.date_range(
    start=idx[-1] + pd.offsets.QuarterEnd(),
    periods=future_steps,
    freq='QE'
)

#Wykres AR(1)
fig_ar1 = go.Figure()

fig_ar1.add_trace(go.Scatter(
    x=idx,
    y=y,
    mode="lines",
    name="Trend"
))

fig_ar1.add_trace(go.Scatter(
    x=idx[1:],      # pasuje do y_ar1_pred
    y=y_ar1_pred,
    mode="lines",
    name="AR(1) dopasowanie",
    line=dict(color="orange")
))

fig_ar1.add_trace(go.Scatter(
    x=future_dates,
    y=y_future_ar1,
    mode="lines",
    name="AR(1) prognoza 10 lat",
    line=dict(color="yellow", dash="dash")
))

fig_ar1.add_annotation(
    x=0.02, y=0.95, xref="paper", yref="paper",
    showarrow=False, align="left",
    bgcolor="rgba(0,100,0,0.75)", bordercolor="white",
    borderwidth=1, borderpad=4,
    font=dict(color="white", size=14),
    text=eq_text_ar1
)

fig_ar1.update_layout(
    template="plotly_dark",
    width=950,
    height=550,
    title="Model inercyjny AR(1) na TRENDZIE + prognoza 10 lat",
    xaxis_title="Data",
    yaxis_title="Trend"
)

fig_ar1.show()
